In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy import stats

print("=== ENHANCED HORIZON EUROPE MODEL SELECTION ANALYSIS ===")


In [ ]:

# 1. Enhanced Color Settings and Style
SOFT_COLORS = ['#7B9DB8', '#A8C5B0', '#D4B896']
plt.style.use('default')
plt.rcParams.update({
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 15
})



In [ ]:

# 2. File Location Setup
plots_dir = r"D:\\KU Leuven\\Stats\\MDA\\horizon-funding\\data\\plots"
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)
    print("Created directory: {}".format(plots_dir))


In [ ]:

# 3. Load Model Results and Cross-Validation Scores
print("\n=== LOADING MODEL RESULTS ===")
try:
    # Test metrics
    rf_results = catalog.load("rf_test_metrics")
    xgb_results = catalog.load("xgb_test_metrics")
    logistic_results = catalog.load("logistic_test_metrics")
    
    # Cross-validation scores for statistical analysis
    rf_cv_scores = catalog.load("rf_cv_scores")
    xgb_cv_scores = catalog.load("xgb_cv_scores")
    logistic_cv_scores = catalog.load("logistic_cv_scores")
    
    results = {
        'Random Forest': rf_results,
        'XGBoost': xgb_results,
        'Logistic Regression': logistic_results
    }
    
    cv_scores = {
        'Random Forest': rf_cv_scores,
        'XGBoost': xgb_cv_scores,
        'Logistic Regression': logistic_cv_scores
    }
    
    print("All model results and CV scores loaded successfully")
    
except Exception as e:
    print("Error loading results: {}".format(e))


In [ ]:

# 4. Comprehensive Performance Summary with Statistical Analysis
print("\n" + "="*60)
print("COMPREHENSIVE MODEL PERFORMANCE ANALYSIS")
print("="*60)

# Create detailed performance comparison
detailed_summary = []
for model_name, model_results in results.items():
    cv_mean = np.mean(cv_scores[model_name]) if model_name in cv_scores else 0
    cv_std = np.std(cv_scores[model_name]) if model_name in cv_scores else 0
    
    detailed_summary.append({
        'Model': model_name,
        'Test_Accuracy': model_results['accuracy'],
        'Test_F1': model_results['f1'],
        'Test_Precision': model_results['precision'],
        'Test_Recall': model_results['recall'],
        'CV_Mean': cv_mean,
        'CV_Std': cv_std,
        'CV_Stability': 1 - (cv_std / cv_mean) if cv_mean > 0 else 0
    })

detailed_df = pd.DataFrame(detailed_summary)

# Display formatted results
print("\nDETAILED PERFORMANCE METRICS:")
print("-" * 80)
for _, row in detailed_df.iterrows():
    print("{:18s} | Acc: {:.4f} | F1: {:.4f} | Prec: {:.4f} | Rec: {:.4f}".format(
        row['Model'], row['Test_Accuracy'], row['Test_F1'], row['Test_Precision'], row['Test_Recall']))
    print("{:18s} | CV: {:.4f}±{:.4f} | Stability: {:.4f}".format(
        "", row['CV_Mean'], row['CV_Std'], row['CV_Stability']))
    print("-" * 80)


In [ ]:

# 5. Statistical Significance Testing
print("\n=== STATISTICAL SIGNIFICANCE ANALYSIS ===")
if len(cv_scores) == 3:
    models = list(cv_scores.keys())
    comparisons = []
    
    for i in range(len(models)):
        for j in range(i+1, len(models)):
            model1, model2 = models[i], models[j]
            t_stat, p_value = stats.ttest_rel(cv_scores[model1], cv_scores[model2])
            
            significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
            
            comparisons.append({
                'Comparison': "{} vs {}".format(model1, model2),
                'T-statistic': t_stat,
                'P-value': p_value,
                'Significance': significance
            })
            
            print("{} vs {}:".format(model1, model2))
            print("  T-statistic: {:.4f}, P-value: {:.4f} ({})".format(t_stat, p_value, significance))

models = list(results.keys())


In [ ]:

# 6. Figure 1: Performance Metrics Comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Model Performance Metrics Comparison', fontsize=16, fontweight='bold', y=0.95)

metrics = ['accuracy', 'f1', 'precision', 'recall']
metric_titles = ['Accuracy', 'F1 Score', 'Precision', 'Recall']

for idx, (metric, title) in enumerate(zip(metrics, metric_titles)):
    row, col = idx // 2, idx % 2
    values = [results[model][metric] for model in models]
    
    bars = axes[row, col].bar(models, values, color=SOFT_COLORS, alpha=0.8, 
                             edgecolor='white', linewidth=2)
    axes[row, col].set_title(title, fontweight='bold', pad=15)
    axes[row, col].set_ylabel('Score', fontweight='bold')
    axes[row, col].set_ylim(0, 1.1)
    axes[row, col].tick_params(axis='x', rotation=15)
    axes[row, col].grid(True, alpha=0.3)
    
    for bar, val in zip(bars, values):
        axes[row, col].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                           '{:.3f}'.format(val), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.subplots_adjust(top=0.9, hspace=0.3, wspace=0.3)
plt.savefig("{}/performance_metrics_comparison.png".format(plots_dir), 
           dpi=300, bbox_inches='tight', facecolor='white')
print("Performance metrics plot saved to: {}/performance_metrics_comparison.png".format(plots_dir))
plt.show()


In [ ]:

# 7. Figure 2: Cross-Validation Stability Analysis
fig, ax = plt.subplots(figsize=(12, 8))
cv_means = [np.mean(cv_scores[model]) for model in models]
cv_stds = [np.std(cv_scores[model]) for model in models]

bars = ax.bar(models, cv_means, yerr=cv_stds, capsize=8, 
              color=SOFT_COLORS, alpha=0.8, edgecolor='white', linewidth=2)
ax.set_title('Cross-Validation Performance and Stability', fontsize=16, fontweight='bold', pad=20)
ax.set_ylabel('CV Accuracy', fontweight='bold')
ax.tick_params(axis='x', rotation=15)
ax.grid(True, alpha=0.3)

for bar, mean, std in zip(bars, cv_means, cv_stds):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.01,
            '{:.3f}'.format(mean), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig("{}/cv_stability_analysis.png".format(plots_dir), 
           dpi=300, bbox_inches='tight', facecolor='white')
print("CV stability plot saved to: {}/cv_stability_analysis.png".format(plots_dir))
plt.show()


In [ ]:

# 8. Figure 3: Confusion Matrices
class_labels = ['Small\n(<=2M EUR)', 'Medium\n(2-4M EUR)', 'Large\n(>4M EUR)']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Confusion Matrices for Model Comparison', fontsize=16, fontweight='bold', y=0.95)

for idx, (model_name, model_results) in enumerate(results.items()):
    if 'confusion_matrix' in model_results:
        cm = np.array(model_results['confusion_matrix'])
        
        cmap = sns.blend_palette(['white', SOFT_COLORS[idx]], as_cmap=True)
        
        sns.heatmap(cm, annot=True, fmt='d', cmap=cmap,
                   xticklabels=class_labels, yticklabels=class_labels,
                   ax=axes[idx], cbar_kws={'shrink': 0.8},
                   linewidths=1, linecolor='white',
                   annot_kws={'fontsize': 12, 'fontweight': 'bold'})
        
        axes[idx].set_title('{}'.format(model_name), fontweight='bold', pad=20)
        axes[idx].set_xlabel('Predicted Class', fontweight='bold')
        axes[idx].set_ylabel('True Class', fontweight='bold')

plt.tight_layout()
plt.subplots_adjust(top=0.85, wspace=0.3)
plt.savefig("{}/confusion_matrices_comparison.png".format(plots_dir), 
           dpi=300, bbox_inches='tight', facecolor='white')
print("Confusion matrices plot saved to: {}/confusion_matrices_comparison.png".format(plots_dir))
plt.show()


In [ ]:

# 9. Figure 4: Performance Radar Chart
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

radar_metrics = ['accuracy', 'f1', 'precision', 'recall']
angles = np.linspace(0, 2*np.pi, len(radar_metrics), endpoint=False).tolist()
angles += angles[:1]

for i, (model_name, color) in enumerate(zip(models, SOFT_COLORS)):
    values = [results[model_name][metric] for metric in radar_metrics]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=4, label=model_name, 
            color=color, alpha=0.8, markersize=8)
    ax.fill(angles, values, alpha=0.15, color=color)

ax.set_xticks(angles[:-1])
ax.set_xticklabels([m.capitalize() for m in radar_metrics], fontsize=12, fontweight='bold')
ax.set_ylim(0, 1)
ax.set_title('Overall Performance Radar Chart', fontweight='bold', pad=30, fontsize=16)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("{}/performance_radar_chart.png".format(plots_dir), 
           dpi=300, bbox_inches='tight', facecolor='white')
print("Performance radar chart saved to: {}/performance_radar_chart.png".format(plots_dir))
plt.show()


In [ ]:

# 10. Advanced Model Selection Logic
print("\n" + "="*60)
print("ADVANCED MODEL SELECTION ANALYSIS")
print("="*60)

weighting_schemes = {
    'Balanced': {'accuracy': 0.25, 'f1': 0.25, 'precision': 0.25, 'recall': 0.25},
    'F1-Focused': {'accuracy': 0.2, 'f1': 0.4, 'precision': 0.2, 'recall': 0.2},
    'Precision-Focused': {'accuracy': 0.2, 'f1': 0.2, 'precision': 0.4, 'recall': 0.2},
    'Recall-Focused': {'accuracy': 0.2, 'f1': 0.2, 'precision': 0.2, 'recall': 0.4}
}

selection_results = {}

for scheme_name, weights in weighting_schemes.items():
    scores = {}
    for model_name, model_results in results.items():
        composite_score = sum(model_results[metric] * weight 
                            for metric, weight in weights.items())
        stability_bonus = detailed_df[detailed_df['Model'] == model_name]['CV_Stability'].iloc[0] * 0.1
        final_score = composite_score + stability_bonus
        scores[model_name] = final_score
    
    best_model = max(scores.items(), key=lambda x: x[1])
    selection_results[scheme_name] = best_model
    
    print("\n{} WEIGHTING SCHEME:".format(scheme_name.upper()))
    print("Weights: {}".format(weights))
    for model, score in sorted(scores.items(), key=lambda x: x[1], reverse=True):
        print("  {:18s}: {:.4f}".format(model, score))
    print("Selected: {} (Score: {:.4f})".format(best_model[0], best_model[1]))


In [ ]:

# 11. Business Context Analysis
print("\n" + "="*60)
print("BUSINESS CONTEXT ANALYSIS")
print("="*60)

print("\nFOR EU FUNDING PREDICTION TASK:")
print("\n1. PRECISION IMPORTANCE:")
print("   High precision reduces false positive funding predictions")
print("   Prevents overestimation of funding opportunities")
print("   Critical for resource allocation planning")

print("\n2. RECALL IMPORTANCE:")
print("   High recall ensures funding opportunities are not missed")
print("   Important for comprehensive funding landscape analysis")
print("   Helps identify all potential funding sources")

print("\n3. MODEL INTERPRETABILITY:")
print("   Random Forest: High - Can extract feature importance")
print("   XGBoost: Medium - Provides feature importance but complex")
print("   Logistic Regression: Highest - Direct coefficient interpretation")

print("\n4. COMPUTATIONAL EFFICIENCY:")
print("   Logistic Regression: Fastest training and prediction")
print("   Random Forest: Moderate - Parallelizable")
print("   XGBoost: Slower but optimized")


In [ ]:

# 12. Final Recommendation
print("\n" + "="*60)
print("FINAL MODEL SELECTION RECOMMENDATION")
print("="*60)

model_votes = {}
for scheme, (model, score) in selection_results.items():
    model_votes[model] = model_votes.get(model, 0) + 1

most_voted = max(model_votes.items(), key=lambda x: x[1])

print("\nRECOMMENDED MODEL: {}".format(most_voted[0]))
print("Selected by {}/{} weighting schemes".format(most_voted[1], len(weighting_schemes)))

recommended_model = most_voted[0]
recommended_results = results[recommended_model]

print("\n{} PERFORMANCE SUMMARY:".format(recommended_model.upper()))
print("   Test Accuracy: {:.4f}".format(recommended_results['accuracy']))
print("   F1 Score: {:.4f}".format(recommended_results['f1']))
print("   Precision: {:.4f}".format(recommended_results['precision']))
print("   Recall: {:.4f}".format(recommended_results['recall']))

if recommended_model in cv_scores:
    cv_mean = np.mean(cv_scores[recommended_model])
    cv_std = np.std(cv_scores[recommended_model])
    print("   Cross-Val Mean: {:.4f} ± {:.4f}".format(cv_mean, cv_std))

print("\nSELECTION RATIONALE:")
if recommended_model == 'Random Forest':
    print("   Excellent balance of performance across all metrics")
    print("   High interpretability through feature importance")
    print("   Robust to overfitting through ensemble approach")
    print("   Handles mixed data types effectively")
    print("   Good stability in cross-validation")
elif recommended_model == 'XGBoost':
    print("   Superior predictive performance")
    print("   Advanced gradient boosting optimization")
    print("   Good handling of complex patterns")
    print("   Built-in regularization")
elif recommended_model == 'Logistic Regression':
    print("   Highest interpretability")
    print("   Fast training and prediction")
    print("   Good baseline performance")
    print("   Minimal computational requirements")

print("\nMODEL SELECTION COMPLETE!")
print("Ready for deployment in EU funding prediction system")


In [ ]:

# 13. Export Selection Summary
export_data = []
for model_name, model_results in results.items():
    export_data.append({
        'Model': model_name,
        'Selected': model_name == recommended_model,
        'Accuracy': model_results['accuracy'],
        'F1_Score': model_results['f1'],
        'Precision': model_results['precision'],
        'Recall': model_results['recall'],
        'CV_Mean': np.mean(cv_scores.get(model_name, [0])),
        'CV_Std': np.std(cv_scores.get(model_name, [0]))
    })

export_df = pd.DataFrame(export_data)
export_path = "{}/model_selection_summary.csv".format(plots_dir)
export_df.to_csv(export_path, index=False)
print("\nSelection summary exported to: {}".format(export_path))

print("\nAll files saved to: {}".format(plots_dir))
print("Generated files:")
print("  - performance_metrics_comparison.png")
print("  - cv_stability_analysis.png") 
print("  - confusion_matrices_comparison.png")
print("  - performance_radar_chart.png")
print("  - model_selection_summary.csv")
print("="*60)